## Goal

* use the mathematical formulation of the optimization task,
* formulate the indicator function and explain how it relates to the outcome
of the classification,
* predict and explain the outcome of using different kernels,
* explain the effect of the C-value when using slack variables.

In [1]:
import numpy as np, random, math, scipy
from scipy.optimize import minimize
import matplotlib.pyplot as plt
%matplotlib inline

In [239]:
k_linear = lambda x, y: np.dot(x, y)

In [371]:
def objective(inner, a):
    return .5 * (a * inner).sum() - a.sum()

In [378]:
def zerofun(t, a):
    return np.dot(t, a)

In [373]:
from functools import partial

In [451]:
def calc(k, x, t):
    N = len(x)
    start = np.zeros(N)
    bounds=[(0, None) for b in range(N)]
    constraint={'type':'eq', 'fun':partial(zerofun, t)}
    p = np.einsum('i,j->ij', t, t) * np.array([[k(m, n) for m in x] for n in x])
    cached_obj = partial(objective, p)
    return minimize(cached_obj, start, bounds=bounds, constraints=constraint)

In [454]:
class_a = np.concatenate((np.random.randn(10, 2) * 0.2 + [1.5, 0.5], np.random.randn(10, 2) * 0.2 + [-1.5, 0.5]))
class_b = np.random.randn(20, 2) * 0.2 + [0.0, -0.5]
inputs = np.concatenate((class_a, class_b))
targets = np.concatenate((np.ones(class_a.shape[0]), -np.ones(class_b.shape[0])))
N = inputs.shape[0]
permute = list(range(N))
random.shuffle(permute)
inputs = inputs[permute, :]
targets = targets[permute]

In [455]:
res = calc(k_linear, inputs, targets)
res

     fun: -78371573.32194784
     jac: array([ 6.,  6.,  4.,  6.,  2., 12.,  0.,  8.,  8., 10.,  2., 12.,  8.,
        2.,  2.,  2., 12.,  4.,  6.,  6.,  8.,  8.,  2.,  4.,  4., 10.,
       -2.,  6.,  2.,  6.,  8.,  4., -2.,  4.,  0.,  6.,  8.,  4.,  6.,
        6.])
 message: 'Optimization terminated successfully.'
    nfev: 985
     nit: 22
    njev: 22
  status: 0
 success: True
       x: array([6.40551289e-11, 2.58181652e-13, 2.78298316e-13, 4.84141741e-08,
       1.10512769e-12, 1.01027844e-12, 3.83306061e-13, 5.59455748e-13,
       1.45094987e-12, 7.96728680e-13, 1.04091513e+08, 3.11330484e-13,
       1.63035881e-12, 9.76248537e+07, 0.00000000e+00, 3.06830260e-13,
       1.29230910e-12, 2.15141977e-13, 7.79735189e-13, 1.25403621e-12,
       1.62652387e-12, 1.22519022e-10, 4.59221345e-13, 1.19286549e+07,
       1.42123867e-07, 9.29863357e-13, 2.59296006e+00, 4.23770465e-13,
       1.78493422e-07, 8.69771571e-13, 1.14941547e-12, 8.06577014e-13,
       1.83953165e+07, 7.36213610e-13

In [462]:
a = res['x']
indices = np.where(a > 10**-5)

In [474]:
inputs[indices][0]

array([-1.57079483,  0.11983581])

In [481]:
b = a[indices] * targets[indices] * [k_linear(inputs[indices][0], inputs[i]) for i in indices[0]]
b

array([ 2.58329843e+08, -1.22365049e+07, -2.45672822e+07, -6.59699476e+00,
        6.24966134e+06])

array([ 2.58329843e+08, -1.22365049e+07, -2.45672822e+07, -6.59699476e+00,
        6.24966134e+06])